### Webscrapping with Pandas pd.read_html()  + ODBC on SQL Server with pyodbc

#### Source : Wikipedia / Liste des pays par Temperature

In [1]:
import numpy as np
import pandas as pd
# Beautifulsoup already installed into Anaconda environment

# Librairy PyODBC for SQL Server connection
import os
import pyodbc
# setup SQL Server connection session
server = '-PCSJN\DATAVIZ'  
database = 'Climbing_Staging'
connection_string= (
                    'DRIVER={SQL Server};SERVER='
                     + server
                     + ';DATABASE='
                     + database 
                     + ';Trusted_Connection=yes'                  
                    )

# Function for injecting dataframe content into SQL Server Table
def template_SQL_insert_values(table_name,dict):
    
    columns = "("
    for k in dict.keys():
        columns += str(k) + ','
    columns = columns[:-1] ### remove last comma
    columns += ")"
    
    req = f"""INSERT INTO {table_name} {columns} VALUES {tuple(dict.values())}"""
    return req

In [2]:
### 1er url
Wiki_Countries_Mean_Temperature = "https://fr.wikipedia.org/wiki/Liste_des_pays_par_temp%C3%A9rature_moyenne"
### récuperer toutes les tables html vu sur le site (url)
Dataset_Wiki_Countries_Mean_Temperature = pd.read_html(Wiki_Countries_Mean_Temperature)
type(Dataset_Wiki_Countries_Mean_Temperature), len(Dataset_Wiki_Countries_Mean_Temperature)

(list, 3)

In [3]:
Dataset_Wiki_Countries_Mean_Temperature[0]

,Rang,Pays,Température annuelle moyenne (1961–1990) en °C
0,1,Burkina Faso,2825
1,1,Mali,2825
2,3,Kiribati,2820
3,4,Djibouti,2800
4,4,Tuvalu,2800
...,...,...,...
184,185,Kirghizistan,155
185,186,Norvège,150
186,187,Mongolie,"−0,70"
187,188,Russie,"−5,10"


In [4]:
Dataset_Wiki_Countries_Mean_Temperature[0].columns

Index(['Rang', 'Pays', 'Température annuelle moyenne (1961–1990) en °C'], dtype='object')

In [5]:
Dataset_Wiki_Countries_Mean_Temperature[0].columns = ['Rang', 'Pays', 'Temperature']

In [6]:
Dataset_Wiki_Countries_Mean_Temperature[0]

,Rang,Pays,Temperature
0,1,Burkina Faso,2825
1,1,Mali,2825
2,3,Kiribati,2820
3,4,Djibouti,2800
4,4,Tuvalu,2800
...,...,...,...
184,185,Kirghizistan,155
185,186,Norvège,150
186,187,Mongolie,"−0,70"
187,188,Russie,"−5,10"


In [7]:
Dataset_Wiki_Countries_Mean_Temperature[0].dtypes

Rang            int64
Pays           object
Temperature    object
dtype: object

Attention A CORRIGER !!!! : pour les valeurs positives Uniquement : le séparateur de décimales ( , ) est perdue pdt extraction


In [11]:
Wiki_Climate_perCountries_Mean_Temperature_Df0 = Dataset_Wiki_Countries_Mean_Temperature[0]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_Climate_perCountries_Mean_Temperature_Df0
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Rang INT,
                                        Pays VARCHAR(200),
                                        Temperature VARCHAR(50)                                                                                                                                              
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_Climate_perCountries_Mean_Temperature_Df0.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_Climate_perCountries_Mean_Temperature_Df0',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

err: ('42S22', "[42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid column name 'Côte d'Ivoire'. (207) (SQLExecDirectW)")


In [9]:
Dataset_Wiki_Countries_Mean_Temperature[1]

,v · mTemps (météorologie),v · mTemps (météorologie).1
0,Précipitations,Averse Bruine Bruine verglaçante Giboulée Grêl...
1,Phénomènes atmosphériques violents étendus,Blizzard Bourrasque de neige Brouillard Cyclon...
2,Autres phénomènes météorologiques,Arc-en-ciel Canicule Ensoleillement Gel Dégel ...
3,Prévision,Alerte Alerte cyclonique Avis de coup de vent ...
4,Climats,Climatologie Microclimat Records climatiques R...
5,Saisons,Automne Été Hiver Mousson Printemps Saison hum...
6,Types de météorologies,Aéronautique Agricole Biométéorologie Espace E...
7,Glossaire de la météorologie,Glossaire de la météorologie


In [10]:
Dataset_Wiki_Countries_Mean_Temperature[2]

,v · mListes de pays,v · mListes de pays.1
0,Démographie,Population 1700 1800 1907 Densité de populatio...
1,Économie,Monnaies Dette publique Dette extérieure Égali...
2,Éducation,Dépenses d'éducation Taux d'alphabétisation Cl...
3,Environnement,Performance environnementale Empreinte écologi...
4,Géographie,Continent Superficie forestière Sans cours d'e...
5,Militaire,Paix Dépenses militaires Forces militaires Ne ...
6,Politique,États membres de l'ONU Date d'acquisition de l...
7,Religion,Par religion Irréligion Bouddhisme Christianis...
8,Santé,Dépenses de santé Consommation de médicaments ...
9,Droit,Peine de mort exécutions les plus récentes Tau...
